In [1]:
import time
import pandas as pd
from datetime import datetime
import threading

import bernielib as bl

bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [2]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [3]:
ber.robotHome()
ber.pipetteHome()

# Refill tips

In [27]:
ber.tips_rack.refill()

# Recalibrate pipette tip rack

In [40]:
ber.calibrateRack(rack='tips')

(76.6, 149.765, 168.4)

In [41]:
ber.move(z=50)

# Preparing DNA for the experiment

## Preparing fake DNA sample, only for testing. Don't run at a real run

In [9]:
dna_500 = bl.createSample('eppendorf', 'dna_500', ber.samples_rack, 0, 10, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 49000)

In [10]:
ber.pickUpNextTip()

In [11]:
ber.move(z=50)

In [12]:
ber.transferLiquid(water, dna_500, 50)

In [13]:
ber.move(z=50)

In [14]:
ber.dumpTipToWaste()

## Preparing diluted stock solution

In [7]:
dna_stock = bl.createSample('eppendorf', 'dna_stock', ber.samples_rack, 0, 9, 160)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 600)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 40000)

In [6]:
ber.pickUpNextTip()

In [7]:
ber.move(z=50)

In [8]:
ber.transferLiquid(water, dna_stock_2, 980)

In [9]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [10]:
ber.transferLiquid(dna_stock, dna_stock_2, 20, v_immerse_dispense=1600)

In [11]:
ber.mixByScript(dna_stock_2, script)

In [12]:
ber.pipetteHome()

In [ ]:
ber.dumpTipToWaste()

## Preparing sample solutions with DNA ladder

In [57]:
ber.pickUpNextTip()

In [58]:
samples_list = bl.createSamplesToPurifyList(ber, [0])

In [59]:
for sample in samples_list:
    ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)

In [60]:
ber.move(z=50)

In [61]:
ber.dumpTipToWaste()

In [62]:
ber.pipetteHome()

# Running a protocol

In [8]:
def getSampleTopZ(sample, rack):
    z_above_rack = sample.getZAboveRack()
    z_rack_top = rack.getZ()
    return z_rack_top - z_above_rack

In [9]:
def calcBeadsVolumeToAdd(robot, samples_list, cutoff_list):
    v_beads_list = []
    for sample, dna_size in zip(samples_list, cutoff_list):
        v_beads = robot.calcBeadsVol(sample, dna_size)
        v_beads_list.append(v_beads)
    return v_beads_list

In [10]:
def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

In [11]:
def waitAndMixByScript(robot, sample, timestamp, delay, mix_script, tip_col, tip_row):
    new_ts = time.time()
    robot.pickUpTip(tip_col, tip_row)
    robot.move(z=50)
    while (new_ts - timestamp) < delay:
        robot.mixByScript(sample, mix_script)
        robot.move(z=50)
        new_ts = time.time()
    robot.dumpTipToPosition(tip_col, tip_row)

In [12]:
def addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario, z_safe=50):
    robot.move(z=z_safe)
    #robot.pickUpNextTip()
    #robot.move(z=z_safe)
    # Mixing beads before experiment
    #robot.mixByScript(beads, beads_mix_scenario)
    # Transferring beads to sample
    robot.transferLiquid(beads, sample, v_beads)
    # Mixing sample with beads
    robot.mixByScript(sample, sample_mix_scenario)
    robot.move(z=z_safe)
    #robot.dumpTipToWaste()
    robot.move(z=z_safe)

In [13]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        #robot.dumpTipToWaste()
        robot.returnTipBack()
    
    return timestamp

In [14]:
def removeSupernatant(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 200:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    pipetteThread = threading.Thread(target=robot.movePipetteToVolume, args=(250,))
    pipetteThread.start()
    robot.moveToSample(sample)
    #robot.movePipetteToVolume(250)
    robot.move(z=z0)
    pipetteThread.join()
    robot.movePipetteToVolume(50)
    time.sleep(delay)
    robot.move(z=z0-0.5)
    time.sleep(delay)
    robot.moveAxisDelta('X', 1.5)
    robot.movePipetteToVolume(40)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', -1.5)
    robot.moveAxisDelta('Y', 1.5)
    robot.movePipetteToVolume(30)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('Y', -1.5)
    robot.moveAxisDelta('X', -1.5)    
    robot.movePipetteToVolume(20)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', 1.5)
    robot.moveAxisDelta('Y', -1.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay/4.0)
    robot.move(z=z_safe)
    robot.dispenseLiquid(waste, 250, blow_extra=True)
    robot.move(z=z_safe)
    robot.dumpTipToWaste()

In [15]:
def removeSupernatantFast(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 0:
        pipetteThread = threading.Thread(target=robot.movePipetteToVolume, args=(200,))
        pipetteThread.start()
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        pipetteThread.join()
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        time.sleep(delay)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    robot.dumpTipToWaste()

In [16]:
def removeSupernatantAllSamples(robot, samples_list, waste, how='fast'):
    counter = 0
    for sample in samples_list:
        if how == 'fast':
            removeSupernatantFast(robot, sample, waste)
        else:
            removeSupernatant(robot, sample, waste)
        if counter == 0:
            sample_dried_timestamp = time.time()
        counter += 1
    return sample_dried_timestamp

In [17]:
def add80PctEthanol(robot, samples_list, ethanol, volume, z_safe=50):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    
    counter = 0
    for sample in samples_list:
        robot.transferLiquid(ethanol, sample, volume, touch_wall=False)
        if counter == 0:
            ethanol_added_time = time.time()
        counter += 1
    
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    
    return ethanol_added_time

In [18]:
def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    # Washing steps, moving along the wall
    # 1
    z_curr = z_top + 24
    #z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.0)
    robot.movePipetteToVolume(volume/4.0)
    time.sleep(delay/4.0)
    # 2
    z_curr = z_top + 28
    #z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    #robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(2 * (volume/4.0))
    time.sleep(delay/4.0)
    # 3
    z_curr = z_top + 32
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(3 * (volume/4.0))
    time.sleep(delay/4.0)
    # 4
    z_curr = z_top + 36
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(volume)
    time.sleep(delay/4.0)
    
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    robot.movePipetteToVolume(volume+50)
    time.sleep(delay)
    robot.movePipetteToVolume(0)

In [19]:
def elutionMixLrgVol(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    
    # Uptaking
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.8)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    
    # Ejecting liquid
    z_curr = sample.calcAbsLiquidLevelFromVol(1000, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.4)
    robot.movePipetteToVolume(volume+50)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    # To back position
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    
    robot.movePipetteToVolume(0)

In [20]:
#TODO: make safe_z a general property of the robot
def elute(robot, sample, eluent, volume, mix_delay=0.5, mix_times=5, safe_z=50):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMixLrgVol(robot, sample, volume)
        elutionMixLrgVol(robot, sample, volume)
        elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    elutionMix(robot, sample, volume)
    elutionMix(robot, sample, volume)
    
    robot.move(z=safe_z)
    robot.returnTipBack()
    #robot.dumpTipToWaste()
    sample.setVolume(volume)
    return elution_start_time

def eluteAllSamples(robot, samples_list, eluent, V_eluent, mix_delay=0.5, mix_times=4, safe_z=50):
    counter = 0
    for sample in samples_list:
        ts = elute(ber, sample, eluent, V_eluent, mix_delay=mix_delay, mix_times=mix_times, safe_z=safe_z)
        print ()
        if counter == 0:
            elution_start_timestamp = ts
        counter += 1
    return elution_start_timestamp

In [21]:
def separateEluate(robot, eluate_tube, result_tube, pipette_delay=0.5, z_above_bottom=0.5, safe_z=50):
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    # Uptaking liquid
    v = eluate_tube.getVolume()
    robot.moveToSample(eluate_tube)
    robot.movePipetteToVolume(v+5)
    robot.movePipetteToVolume(v)
    z0 = robot._getTubeZBottom(eluate_tube)
    robot.move(z=z0-z_above_bottom)
    robot.movePipetteToVolume(0)
    time.sleep(pipette_delay)
    # Dispensing liquid
    robot.moveToSample(result_tube)
    robot.movePipetteToVolume(v+5)
    time.sleep(pipette_delay)
    robot.movePipetteToVolume(200)
    time.sleep(pipette_delay)
    robot.touchWall(result_tube)
    robot.movePipetteToVolume(0)
    robot.move(z=safe_z)
    ber.dumpTipToWaste()

In [22]:
def oneStagePurification(robot, N_samples, initial_vol_list, cutoff_list, V_avail_beads, V_avail_water, V_avail_ethanol, 
                         beads_rack, beads_col, beads_row, sample_mix_scenario, beads_mix_scenario,
                         V_waste=0, T_absorb=600, T_pull=60, T_wash=30, T_dry=300, T_elute=600,
                         V_wash=200, V_elute=30):
    
    print("Experiment started ", datetime.now().strftime("%H:%M:%S"))
    samples_list = bl.createSamplesToPurifyList(robot, initial_vol_list)
    result_list = bl.createPurifiedSamplesList(robot, N_samples)
    beads = bl.createSample('eppendorf', 'beads', beads_rack, beads_col, beads_row, V_avail_beads)
    waste = bl.createSample('50ml', 'liquid_waste', robot.reagents_rack, 0, 1, V_waste)
    water = bl.createSample('50ml', 'water', robot.reagents_rack, 1, 1, V_avail_water)
    EtOH80pct = bl.createSample('5ml', 'EtOH80pct', robot.reagents_rack, 0, 0, V_avail_ethanol)
    
    v_beads_list = calcBeadsVolumeToAdd(robot, samples_list, cutoff_list=cutoff_list)
    # Adding beads
    timestamp_beads_added = addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario)
    print("Beads added ", datetime.now().strftime("%H:%M:%S"))
    print("Waiting for DNA absorption")
    last_tip_col, last_tip_row = robot.last_tip_coord
    waitAndMixByScript(robot, samples_list[0], timestamp_beads_added, T_absorb, sample_mix_scenario, 
                       last_tip_col, last_tip_row)
    #waitAfterTimestamp(timestamp_beads_added, T_absorb)
    print("DNA absorption finished ", datetime.now().strftime("%H:%M:%S"))
    # Removing supernatant
    robot.moveMagnetsTowardsTube(poweroff=True)
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Supernatant removed ", datetime.now().strftime("%H:%M:%S"))
    # Ethanol wash
    # Wash 1
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    print("Wash 1: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    print("Wash 1: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Wash 1: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Wash 2
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    print("Wash 2: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    print("Wash 2: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    timestamp_ethanol_removed = removeSupernatantAllSamples(robot, samples_list, waste, how='full')
    print("Wash 2: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Drying ethanol
    waitAfterTimestamp(timestamp_ethanol_removed, T_dry)
    print("Ethanol drying finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Elution
    # Adding water
    elution_start_timestamp = eluteAllSamples(robot, samples_list, water, V_elute, mix_delay=0.5, mix_times=6, safe_z=50)
    print("Eluent added ", datetime.now().strftime("%H:%M:%S"))
    last_tip_col, last_tip_row = robot.last_tip_coord
    waitAndMixByScript(robot, samples_list[0], elution_start_timestamp, T_absorb, sample_mix_scenario, 
                       last_tip_col, last_tip_row)
    #waitAfterTimestamp(elution_start_timestamp, T_elute) # elution wait
    print("Elution incubation finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Magnetic beads to the side of the tubes
    ber.moveMagnetsTowardsTube()
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    # Moving liquid to the resulting tubes
    for sample, result in zip(samples_list, result_list):
        separateEluate(ber, sample, result)
    print("Eluate transferred to the new tube ", datetime.now().strftime("%H:%M:%S"))
    print("Experiment finished ", datetime.now().strftime("%H:%M:%S"))

In [23]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [24]:
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 300)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 36000)

In [43]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [48]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[150],
                     V_avail_beads=500, V_avail_water=40000, V_avail_ethanol=4500, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=120, T_elute=480)

Experiment started  14:51:41
Beads added  14:53:47
Waiting for DNA absorption
DNA absorption finished  15:01:50
Beads pulled to the side  15:02:52
Supernatant removed  15:03:22
Wash 1: ethanol added  15:03:55
Wash 1: ethanol incubation finished  15:04:16
Wash 1: ethanol removed  15:04:48
Wash 2: ethanol added  15:05:21
Wash 2: ethanol incubation finished  15:05:42
Wash 2: ethanol removed  15:06:18
Ethanol drying finished  15:08:19

Eluent added  15:11:09
Elution incubation finished  15:19:05
Beads pulled to the side  15:20:06
Eluate transferred to the new tube  15:20:36
Experiment finished  15:20:36


In [49]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [50]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[250],
                     V_avail_beads=430, V_avail_water=38000, V_avail_ethanol=4100, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=120, T_elute=480)

Experiment started  15:24:18
Beads added  15:26:23
Waiting for DNA absorption
DNA absorption finished  15:34:44
Beads pulled to the side  15:35:46
Supernatant removed  15:36:16
Wash 1: ethanol added  15:36:49
Wash 1: ethanol incubation finished  15:37:10
Wash 1: ethanol removed  15:37:42
Wash 2: ethanol added  15:38:15
Wash 2: ethanol incubation finished  15:38:36
Wash 2: ethanol removed  15:39:12
Ethanol drying finished  15:41:13

Eluent added  15:44:03
Elution incubation finished  15:52:00
Beads pulled to the side  15:53:01
Eluate transferred to the new tube  15:53:31
Experiment finished  15:53:31


In [28]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [29]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[450],
                     V_avail_beads=350, V_avail_water=36000, V_avail_ethanol=4500, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=120, T_elute=480)

Experiment started  16:52:24
Beads added  16:54:27
Waiting for DNA absorption
DNA absorption finished  17:02:37
Beads pulled to the side  17:03:39
Supernatant removed  17:04:08
Wash 1: ethanol added  17:04:42
Wash 1: ethanol incubation finished  17:05:03
Wash 1: ethanol removed  17:05:34
Wash 2: ethanol added  17:06:08
Wash 2: ethanol incubation finished  17:06:29
Wash 2: ethanol removed  17:07:04
Ethanol drying finished  17:09:04

Eluent added  17:11:55
Elution incubation finished  17:19:50
Beads pulled to the side  17:20:52
Eluate transferred to the new tube  17:21:21
Experiment finished  17:21:21


In [30]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [31]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[1000],
                     V_avail_beads=300, V_avail_water=36000, V_avail_ethanol=4000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=120, T_elute=480)

Experiment started  17:25:33
Beads added  17:27:17
Waiting for DNA absorption
DNA absorption finished  17:35:27
Beads pulled to the side  17:36:28
Supernatant removed  17:36:58
Wash 1: ethanol added  17:37:31
Wash 1: ethanol incubation finished  17:37:52
Wash 1: ethanol removed  17:38:24
Wash 2: ethanol added  17:38:57
Wash 2: ethanol incubation finished  17:39:18
Wash 2: ethanol removed  17:39:53
Ethanol drying finished  17:41:53

Eluent added  17:44:43
Elution incubation finished  17:52:39
Beads pulled to the side  17:53:40
Eluate transferred to the new tube  17:54:09
Experiment finished  17:54:09


# Manual purification

In [32]:
initial_vol_list = [30, 30, 30, 30, 30]
samples_number = len(initial_vol_list)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 300)

In [33]:
ber.pickUpNextTip()
samples_list = bl.createSamplesToPurifyList(ber, [0 for x in range(samples_number)])

for sample, volume in zip(samples_list, initial_vol_list):
    ber.transferLiquid(dna_stock_2, sample, volume, v_immerse_dispense=1600, touch_wall=False)

ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [34]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        robot.dumpTipToWaste()
    
    return timestamp

In [35]:
V_avail_beads = 300 #uL
v_beads_list = [90, 27, 18, 15]
samples_list = bl.createSamplesToPurifyList(ber, [30, 30, 30, 30])
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 11, V_avail_beads)
sample_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')
beads_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')

ber.pickUpNextTip()

ber.move(z=50)
ber.mixByScript(beads, beads_mix_scenario)
ber.move(z=50)

for sample, v_beads in zip(samples_list, v_beads_list):
    ber.transferLiquid(beads, sample, v_beads, touch_wall=False)

ber.move(z=50)
ber.dumpTipToWaste()

In [26]:
ber.move(z=50)

In [27]:
ber.dumpTipToWaste()

# Results analysis

Only sample 3 made by the robot got some weak bands.

Rumors that NEB ladder does not work with beads; no size selection

Kapa beads gel has a ladder on one line, than input (different than ladder) on the other line. 

# Next steps

1. Get different company ladder
2. Make custom ladder from linear fragments that I already have.